In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import json
import fileinput
from tqdm import tqdm
import string
import re
import json
from os import walk
import os
import ast
import re

data_dir = "/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/sub/structured_data/"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [2]:
wiki_team = pd.DataFrame()
with fileinput.input(files="../" + "wikipedia_crawler/team_info_wikipedia.jl") as file:
    for line in file:
        conv = json.loads(line)
        wiki_team = wiki_team.append(conv, ignore_index=True)
wiki_team.head()

,wiki_team_url,name,Fullname,Nickname(s),Founded,Ground,Capacity,Owner,Chairman,Manager,...,HeadCoach,CEO,Owner(s),Joint-Chairman,Co-chairmen,Owners,Stadium,Anear-fullcapacity,Location,Opened
0,https://en.wikipedia.org/wiki/Wigan_Athletic,Wigan Athletic,[WiganAthleticFootballClub],"[TheLatics, TheTics]","[1932, ;89 yearsago, (, 1932, )]",[DWStadium],25138,[Phoenix2021Limited],TalalAlHammad,[LeamRichardson],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.wikipedia.org/wiki/Tottenham_Hotspur,Tottenham Hotspur,[TottenhamHotspurFootballClub],[TheLilywhites],"[5 September1882, ;139yearsago, (, 1882-09-05...",[TottenhamHotspurStadium],62850,"[ENICInternationalLtd., (85.55%)]",DanielLevy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Birmingham_City,Birmingham City F.C.,[BirminghamCityFootballClub],[Blues],"[1875, ;146 yearsago, (, 1875, ), as, SmallHe...",[StAndrew's],29409,"[BirminghamSportsHoldings75.00%, OrientalRainb...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Wolverhampton_Wa...,Wolverhampton Wanderers,[WolverhamptonWanderersFootballClub],[WolvesTheWanderers],"[1877, ;144 yearsago, (, 1877, ), as, St.Luke...",[MolineuxStadium],32050,[FosunInternational],JeffShi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Sunderland_AFC,Sunderland A.F.C.,[SunderlandAssociationFootballClub],[TheBlackCats],"[1879, ;142 yearsago, (, 1879, ), (as, Sunder...",[StadiumofLight],49000,"[KyrilLouis-Dreyfus, (majority)]",KyrilLouis-Dreyfus,NaN,...,[LeeJohnson],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
world_team = pd.DataFrame()
with fileinput.input(files="../" + "worldfootball_crawler/team_info_worldfootball.jl") as file:
    for line in file:
        conv = json.loads(line)
        world_team = world_team.append(conv, ignore_index=True)
world_team.head()

,name,world_team_url,Country,Founded,stadium,Homepage
0,Liverpool FC,https://www.worldfootball.net/teams/liverpool-fc/,[England],[15/03/1892],[Anfield],[www.liverpoolfc.com/]
1,Tottenham Hotspur,https://www.worldfootball.net/teams/tottenham-...,[England],[05/09/1882],[TottenhamHotspurStadium],[www.spurs.co.uk]
2,Sunderland AFC,https://www.worldfootball.net/teams/sunderland...,[England],[01/01/1879],[StadiumofLight],[www.safc.com]
3,Manchester United,https://www.worldfootball.net/teams/manchester...,[England],[01/03/1878],[OldTrafford],[www.manutd.com/]
4,Wigan Athletic,https://www.worldfootball.net/teams/wigan-athl...,[England],[01/01/1932],[DWStadium],[https//www.wiganathletic.com/]


In [5]:
who_team = pd.DataFrame()
with fileinput.input(files="../" + "whoscored_crawler/team_whoscored.jl") as file:
    for line in file:
        conv = json.loads(line)
        who_team = who_team.append(conv, ignore_index=True)
who_team.head()

,season,name,goals,team_url,shots_per_game,yellow,red,possessions_per,pass_per,Aerialswon,Rating
0,2009/2010,Chelsea,103,https://www.whoscored.com/Teams/15/Show/Englan...,21.9,54,5,56.9,82.5,11.6,7.30
1,2009/2010,Tottenham,67,https://www.whoscored.com/Teams/30/Show/Englan...,17.9,58,1,52.2,76.1,18.2,7.19
2,2009/2010,Manchester United,86,https://www.whoscored.com/Teams/32/Show/Englan...,18.3,49,4,55.6,79.9,13.2,7.19
3,2009/2010,Arsenal,83,https://www.whoscored.com/Teams/13/Show/Englan...,17.4,56,1,57.0,80.8,14.4,7.17
4,2009/2010,Aston Villa,52,https://www.whoscored.com/Teams/24/Show/Englan...,13.1,59,2,48.5,71.8,18.8,7.06


In [6]:
wiki_team["Fullname"] = wiki_team["Fullname"].apply(lambda  x : x[0])

In [7]:
def f(x):
    for idx1, item in enumerate(x):
        for idx, month in enumerate(["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]):
            x[idx1] = re.sub(month, "-" + str(idx+1) + "-", x[idx1]).split(";")[0]
    r = re.compile("\d{1,4}-\d{1,2}-\d{1,4}|\d{4}$|\d{2}-\d{2}-\d{4}|\d{2}-\d{1}-\d{4}|\d{2}-\d{4}")
    filtered_list = list(filter(r.match, x))
    return filtered_list

In [8]:
wiki_team['Founded'] = wiki_team["Founded"].apply(lambda x : set(f([i.lower() for i in x])))

In [9]:
Lev = NormalizedLevenshtein().similarity

In [10]:
wiki_name2idx = dict()
wiki_idx2name = dict()
for idx, name in enumerate(wiki_team["name"]):
    wiki_name2idx[name] = idx
    wiki_idx2name[idx] = name

In [11]:
def f(x):
    tmp = [0,0]
    for name in wiki_team["name"]:
        if Lev(x, name) > 0.3 and Lev(x, name) > tmp[1]:
            tmp[0] = wiki_name2idx[name]
            tmp[1] = Lev(x,name)
    if tmp != [0,0]:
        return tmp[0]

who_team["id"] = who_team["name"].apply(f)
world_team["id"] = world_team["name"].apply(f)


In [12]:
wiki_team["id"] = wiki_team["name"].apply(lambda x : wiki_name2idx[x])

In [13]:
who_team.loc[who_team.name == "Bolton", "id"] = 18

who_team.loc[who_team.name == "Brighton", "id"] = 35

In [14]:
team_merged = wiki_team.merge(world_team, left_on = "id", right_on = 'id', how = "outer")
team_merged = team_merged.merge(who_team[["name", "team_url", "id"]].drop_duplicates().rename({"team_url" : "who_team_url"}, axis = 1), left_on = "id", right_on = 'id', how = "outer")

team_merged.loc[38,"name"] = "Brighton"
team_merged.loc[38, "who_team_url"] = 'https://www.whoscored.com/Teams/211/Show/England-Brighton'
team_merged = team_merged.drop(35, axis = 0)
team_merged.to_csv("../data_/" + "team_join.csv")

In [16]:
match = team_merged[["wiki_team_url", "world_team_url", "who_team_url"]]
match = match.append({"wiki_team_url" : 'https://en.wikipedia.org/wiki/Brentford_F.C.', "world_team_url" : 'https://www.worldfootball.net/teams/brentford-fc/', "who_team_url" : 'https://www.whoscored.com/Teams/189/Show/England-Brentford'}, ignore_index = True)
match["id"] = [i for i in range(len(match))]

# match.loc[38,"who_team_url"] = 'https://www.whoscored.com/Teams/211/Show/England-Brighton'
# match = match.drop(35, axis = 0)

In [17]:
match.to_csv("../data_/" + "team_match.csv", index = None)